In [1]:
# Imports

import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from datetime import datetime

import plotly.plotly as py
import plotly.graph_objs as go
import plotly
plotly.tools.set_credentials_file(username='halima.schede', api_key='0BXIz4i3MnnYF4z7QhA0')

from helpers import *


In [2]:
# Load data
tweets1 = pd.read_csv(access_folder('data') + 'IRAhandle_tweets_1.csv')
tweets2 = pd.read_csv(access_folder('data') + 'IRAhandle_tweets_2.csv')
tweets3 = pd.read_csv(access_folder('data') + 'IRAhandle_tweets_3.csv')
tweets4 = pd.read_csv(access_folder('data') + 'IRAhandle_tweets_4.csv')
tweets5 = pd.read_csv(access_folder('data') + 'IRAhandle_tweets_5.csv')
tweets6 = pd.read_csv(access_folder('data') + 'IRAhandle_tweets_6.csv')
tweets7 = pd.read_csv(access_folder('data') + 'IRAhandle_tweets_6.csv')
tweets8 = pd.read_csv(access_folder('data') + 'IRAhandle_tweets_6.csv')
tweets9 = pd.read_csv(access_folder('data') + 'IRAhandle_tweets_6.csv')



tweets = pd.concat([tweets1, tweets2, tweets3, tweets5, tweets6], axis=0, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)


In [3]:
tweets.describe()

,external_author_id,following,followers,updates,new_june_2018,retweet
count,1.803462e+06,1.803466e+06,1.803466e+06,1.803466e+06,1.803466e+06,1.803466e+06
mean,1.648020e+17,2.755869e+03,6.010060e+03,1.052307e+04,2.444194e-01,4.347368e-01
std,3.361395e+17,4.253591e+03,1.451730e+04,1.980754e+04,4.297425e-01,4.957226e-01
min,3.497640e+07,-1.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00
25%,1.930748e+09,2.510000e+02,2.590000e+02,1.726000e+03,0.000000e+00,0.000000e+00
50%,2.577152e+09,1.161000e+03,9.480000e+02,4.207000e+03,0.000000e+00,0.000000e+00
75%,3.312143e+09,3.401000e+03,3.952000e+03,1.154000e+04,0.000000e+00,1.000000e+00
max,9.681310e+17,4.313100e+04,2.512760e+05,1.661130e+05,1.000000e+00,1.000000e+00


In [4]:
tweets.drop(['new_june_2018','harvested_date','content','region','retweet']
            ,axis=1, inplace=True)

In [5]:
tweets.head()

,external_author_id,author,language,publish_date,following,followers,updates,post_type,account_type,account_category
0,9.060000e+17,10_GOP,English,10/1/2017 19:58,1052,9636,253,NaN,Right,RightTroll
1,9.060000e+17,10_GOP,English,10/1/2017 22:43,1054,9637,254,NaN,Right,RightTroll
2,9.060000e+17,10_GOP,English,10/1/2017 22:50,1054,9637,255,RETWEET,Right,RightTroll
3,9.060000e+17,10_GOP,English,10/1/2017 23:52,1062,9642,256,NaN,Right,RightTroll
4,9.060000e+17,10_GOP,English,10/1/2017 2:13,1050,9645,246,RETWEET,Right,RightTroll


In [6]:
# check for NaN values
tweets.isnull().sum()

external_author_id          4
author                      0
language                    0
publish_date                0
following                   0
followers                   0
updates                     0
post_type             1019433
account_type                0
account_category            0
dtype: int64

In [7]:
# convert datetime * respect m/d/y format
tweets['publish_date'] = pd.to_datetime(tweets['publish_date'], format='%m/%d/%Y %H:%M')

### Language Distribution

In [8]:
# create dataframe with publish_date and language
language_time = tweets[['publish_date', 'language']]

In [9]:
def count_language(lang):
    filt = language_time[language_time.language == lang].copy()
    filt['language_num'] = filt.language.map({lang:1}).copy()
    return filt.groupby(pd.Grouper(key='publish_date', freq='1M')).sum()

In [12]:
# Create function that generates the data variable to be used in py.iplot

def trace_generator(lang_df):
    
    data = []
    for lang in lang_df.language.value_counts().index.values[0:10]:
        filtered=count_language(lang)
        strd = pd.Series(filtered.index.strftime('%Y-%m-%d %H-%M-%S'))
        xlabels = list(strd.apply(lambda x: x[0:7]))
        trace = go.Scatter(x=xlabels, 
                            y=filtered.language_num.values,
                            fill='tozeroy',
                            mode= 'none',
                            name=lang)
        data.append(trace)
        
    return data

In [13]:
# english = count_language('English') # create dataframe with appropriate datetimes
# strg = pd.Series(english.index.strftime('%Y-%m-%d %H-%M-%S')) # convert index to strings
# xlabels = list(strg.apply(lambda x: x[0:7])) # list them and crop for month

data = trace_generator(language_time)

layout = go.Layout(
    title='Languages of Tweets by Month',
    xaxis=dict(
        title='Date',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Number of Tweets',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)

In [14]:
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='styling-names')

### Account Category

In [ ]:
russian = count_language('Russian')

In [ ]:
russian.tail(100)